In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
from prog import scheduler as sch

In [ ]:
# save current working directory as homey
# homey = os.path.abspath(os.path.dirname(__file__))
homey = os.getcwd() # works in jupyter notebook

In [ ]:
# set directory paths
dataPath = os.path.join(homey, 'data')
progPath = os.path.join(homey, 'prog')
simPath = os.path.join(homey, 'FB_Sim')
forcPath = os.path.join(simPath, 'ForecastRedoux')
# this should probably be moved out of the simulator and into the main sql area, where that ends up
sqlPath = os.path.join(forcPath, 'SQL')

In [ ]:
# set paths to excel files
forecastFilename = os.path.join(dataPath, 'RegularForecast.xlsx')
mfgCentersFilename = os.path.join(dataPath, 'MfgCenters.xlsx')
moFilename = os.path.join(dataPath, 'MOs.xlsx')
laborAvailFilename = os.path.join(dataPath, 'LaborAvailablePerDay.xlsx')
leadFilename = os.path.join(dataPath, 'LeadTimes.xlsx')

In [ ]:
sys.path.insert(0, forcPath)

In [ ]:
import ForecastMain as fm
import ForecastAPI as fa

In [ ]:
# pull the usual FB_Sim queries
# fa.run_queries(queryPath=sqlPath, dataPath=dataPath)

In [ ]:
# save mfgCenters as df, includes MFG Center assignments and Setup/labor time estimates
mfgCenters = pd.read_excel(mfgCentersFilename, header=0)

In [ ]:
# save current Manufacture Orders
modf = pd.read_excel(moFilename, header=0)

In [ ]:
# save lead time estimates
leadTimes = pd.read_excel(leadFilename, header=0)

In [ ]:
# this is a placeholder for a calculation of start to finish time for a build.
# just using it for earliest schedule date right now.
orderRunTime = 7

In [ ]:
# make a date list with labor availability
dateList = sch.create_date_list(dailyLabor=11)

In [ ]:
# run the auto schedule to get an ideal schedule by priority
moLinesLabor = sch.run_auto_schedule(modf=modf, mfgCenters=mfgCenters, dateList=dateList)

In [ ]:
# use the last scheduled FG in an order to save an ideal schedule
idealSchedule = moLinesLabor.drop_duplicates('ORDER', keep='last')

In [ ]:
# replace the schedule dates on the MO order lines with the new dates for those orders
newMOdf = pd.merge(modf.copy(), idealSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')
newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()
newMOdf.drop(labels='NewDate', axis=1, inplace=True)

In [ ]:
# run the new MO schedule through the FB_Sim to find phantom orders
orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())

In [ ]:
# get a list of phantom orders and their grandparent orders
### Currently phantom orders are placed a day before the order with the shortage will finish.  So
###     if the lead time field is used later, this could throw the phantom schedule dates.  I'll
###     try to avoid this by getting back to the shortage date, but it might become redundant later.
phantoms = orderTimeline[orderTimeline['ITEM'] == 'Phantom'].copy()
buyPhantoms = phantoms[phantoms['Make/Buy'] == 'Buy'].copy()


In [ ]:
# add the lead times to each part
leadPhantoms = pd.merge(buyPhantoms.copy(), leadTimes[['PART','LeadTimes']].copy(), how='left', on='PART')

In [ ]:
# simplify the dataFrame - not necessary
orderLeads = leadPhantoms[['GRANDPARENT','LeadTimes']].copy()

In [ ]:
# sort by grandparent and then lead time to make sure the highest lead time comes first
orderLeads.sort_values(by=['GRANDPARENT','LeadTimes'], ascending=[True, False], inplace=True)

In [ ]:
# drop duplicate grandparents leaving the longest lead as the only one remaining
orderLeads.drop_duplicates('GRANDPARENT', keep='first', inplace=True)

In [ ]:
# rename column for easy merge
orderLeads.rename(columns={'GRANDPARENT':'ORDER'}, inplace=True)

In [ ]:
# adjust the lead time to account for time to place/receive order (2 days) and expected run time for the build
# replace orderRunTime with a calculation of labor needed against average labor available per day
orderLeads['AdjustedLeadTimes'] = orderLeads['LeadTimes'] + 2 + orderRunTime

In [ ]:
# Add comparative dates from dateList
orderLeads = pd.merge(orderLeads.copy(), dateList[['DaysFromStart','StartDate']].copy(), how='left', left_on='AdjustedLeadTimes', right_on='DaysFromStart')

In [ ]:
# rename the start date column to earliest schedule date
### Note: this is info that will need to be unchanged in future loops.  The earliest schedule date should not move in.
orderLeads.rename(columns={'StartDate':'EarliestScheduleDate'}, inplace=True)

In [ ]:
# save a new copy of the modf with longest leads added
leadMOdf = pd.merge(modf.copy(), orderLeads.copy(), how='left', on='ORDER')

In [ ]:
### I don't think this is necessary since I'm dropping columns immediately after
# fill missing leads with 0 since that is effectively what is required
# leadMOdf.fillna(0, inplace=True)

In [ ]:
# drop excess columns
leadMOdf.drop(['LeadTimes','AdjustedLeadTimes','DaysFromStart'], axis=1, inplace=True)

In [ ]:
# dateList['StartDate'].iat[0] > dateList['StartDate'].iat[1]

In [ ]:
# pd.Timestamp.today() - pd.Timedelta('2 days')

In [ ]:
# just temporary
leadMOdfSave = leadMOdf.copy()
mfgCentersSave = mfgCenters.copy()
dateListSave = dateList.copy()

In [ ]:
# refresh!
leadMOdf = leadMOdfSave.copy()
mfgCenters = mfgCentersSave.copy()
dateList = dateListSave.copy()

In [ ]:
# getting a list of Finished Goods on MO's
moFgOnly = leadMOdf[leadMOdf['ORDERTYPE'] == 'Finished Good'].copy()
# sorting by date so the earliest scheduled can be the highest priority
moFgOnly.sort_values('DATESCHEDULED', inplace=True)
# renaming part column to match MO header
mfgCenters.rename(columns={'Part':'PART'}, inplace=True)
# adding centers and labor estimates to MO lines
moLinesLabor = pd.merge(moFgOnly.copy(), mfgCenters.copy(), how='left', on='PART')
# save missing info for later.  Will want user to see what items were missed for lack of data.
missingCenters = moLinesLabor[moLinesLabor['Mfg Center'].isnull()].copy()
missingSetup = moLinesLabor[moLinesLabor['Setup'].isnull()].copy()
missingLabor = moLinesLabor[moLinesLabor['LaborPer'].isnull()].copy()

In [ ]:
# just filling N/A in a way that doesn't throw errors, will need these to be 0 for easy maths
moLinesLabor['SetupTemp'] = moLinesLabor['Setup'].fillna(0)
moLinesLabor['LaborPerTemp'] = moLinesLabor['LaborPer'].fillna(0)
moLinesLabor.drop(['Setup','LaborPer'], axis=1, inplace=True)
moLinesLabor.rename(columns={'SetupTemp':'Setup','LaborPerTemp':'LaborPer'}, inplace=True)

In [ ]:
# replace nulls with 0 for maths.  Probably not necessary, didn't test.
#moLinesLabor.fillna(0, inplace=True)
# create a column for the total labor required for each order
moLinesLabor['LaborRequired'] = moLinesLabor['Setup'] + (moLinesLabor['LaborPer'] * moLinesLabor['QTYREMAINING'])
# calculate cumulative labor needed for builds in their current date order
# moLinesLabor['CumulativeLaborRequired'] = np.nan
# x = 0
# for index in moLinesLabor.index:
#     moLinesLabor.at[index, 'CumulativeLaborRequired'] = moLinesLabor.at[index, 'LaborRequired'].copy() + x
#     x = moLinesLabor.at[index, 'CumulativeLaborRequired'].copy()

In [ ]:
# just temporary
moLinesLaborSave = moLinesLabor.copy()

In [ ]:
# refresh!!
moLinesLabor = moLinesLaborSave.copy()

In [ ]:
# create output schedule dataFrame and a variable to hold total unused labor
outputSchedule = pd.DataFrame(columns=['ORDER','LaborRequired','EarliestScheduleDate','NewDate'])
unusedLabor = 0

In [ ]:
def labor_total(orderLabor, usedLabor, extraLabor):
    totalLabor = orderLabor + usedLabor + extraLabor
    return totalLabor

In [ ]:
lH = 5

In [ ]:
# get a total of the labor needed for the line plus the labor already scheduled and expected to be unused
totalLabor = labor_total(moLinesLabor['LaborRequired'].iat[lH], outputSchedule['LaborRequired'].sum(), unusedLabor)

In [ ]:
# pull the top line from the dateList where the available labor will be enough for the total required labor
tempDate = dateList[dateList['AvailableLabor'] >= totalLabor].head(1)

In [ ]:
# this is the schedule date for the order
schedDate = tempDate['StartDate'].iat[0]

In [ ]:
if schedDate < moLinesLabor['EarliestScheduleDate'].iat[lH]:
    print('k')# move to second priority
else:
    outputSchedule = outputSchedule.append({'ORDER':moLinesLabor['ORDER'].iat[lH],
                                            'LaborRequired':moLinesLabor['LaborRequired'].iat[lH],
                                            'EarliestScheduleDate':moLinesLabor['EarliestScheduleDate'].iat[lH],
                                            'NewDate':schedDate},
                                            ignore_index=True)

In [ ]:
# looping through order lines to auto schedule
# if it comes up with a schedule date before the earliest allowed,
# it will move onto the next line to search for another order that can fit
# 
lH = 0
while lH < len(moLinesLabor):
    # collect labor needed and get relevant schedule date
    totalLabor = labor_total(moLinesLabor['LaborRequired'].iat[lH], outputSchedule['LaborRequired'].sum(), unusedLabor)
    tempDate = dateList[dateList['AvailableLabor'] >= totalLabor].head(1)
    schedDate = tempDate['StartDate'].iat[0]
    if schedDate < moLinesLabor['EarliestScheduleDate'].iat[lH]:
        # if the schedule date is before the line can schedule then move on
        lH+=1
        if lH >= len(moLinesLabor):
            
    else:
        # otherwise add a schedule line to the output and delete the order from the labor list
        outputSchedule = outputSchedule.append({'ORDER':moLinesLabor['ORDER'].iat[lH],
                                                'LaborRequired':moLinesLabor['LaborRequired'].iat[lH],
                                                'EarliestScheduleDate':moLinesLabor['EarliestScheduleDate'].iat[lH],
                                                'NewDate':schedDate},
                                                ignore_index=True)
        moLinesLabor.drop(moLinesLabor.index[lH], inplace=True)
        # set the iterator back to 0 to start back at the top of the remaining priority list
        lH = 0

In [ ]:
outputSchedule

In [ ]:
moLinesLabor.sort_values('EarliestScheduleDate', inplace=True)

In [ ]:
moLinesLabor.index[0]

In [ ]:
moLinesLabor['ScheduledStatus'] = 'unscheduled'

In [ ]:
workingLine = 0

In [ ]:
len(moLinesLabor)

In [ ]:
while workingLine < len(moLinesLabor):
    if moLinesLabor['ScheduledStatus'].iat[workingLine] == 'unscheduled':
        laborNeeded = moLinesLabor['CumulativeLaborRequired'].iat[0]
        tempDateList = dateList[dateList['AvailableLabor'] >= laborNeeded].copy()
        newDate = tempDateList['StartDate'].iat[0]
        if newDate < moLinesLabor['EarliestScheduleDate'].iat[0]:
            skippedLineMarker = workingLine
            
    else:
        workingLine += 1

In [ ]:
moLinesLabor['NewDate'] = np.nan
for index in moLinesLabor.index:
    laborNeeded = moLinesLabor.at[index, 'CumulativeLaborRequired'].copy()
    tempDateList = dateList[dateList['AvailableLabor'] >= laborNeeded].copy()
    newDate = tempDateList['StartDate'].iat[0]
    moLinesLabor.at[index, 'NewDate'] = newDate

In [ ]:
moLinesLabor['NewDate'] = np.nan
for 

In [ ]:
moLinesLabor = sch.run_auto_schedule(modf=leadMOdf, mfgCenters=mfgCenters, dateList=dateList)

In [ ]:
moLinesLabor

In [ ]:
date1 = moLinesLabor['DATESCHEDULED'].iat[0]

In [ ]:
date2 = moLinesLabor['NewDate'].iat[0]

In [ ]:
moLinesLabor.dtypes

In [ ]:
leadMOdf

In [ ]:
modf

In [ ]:
dateList

In [ ]:
pd.Timestamp.today() - pd.Timedelta('2 days')